In [ ]:
%%capture
# Для Collab
# !pip install langchain==1.1.3 langchain-mistralai==1.1.0 langchain-text-splitters==1.0.0 faiss-cpu==1.13.1 mistralai==1.9.11 langchain-community==0.4.1

In [ ]:
%%capture
# Чтобы скачать трейн датасет документов (если не работает, есть ссылка в README)
!wget https://huggingface.co/datasets/Fourzeroo/ITMO-LLM-Course-RAG/resolve/main/questions_data.zip?download=true -O questions_data.zip
!unzip questions_data.zip

In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from typing import List
import json
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
# Положите ключ в .env файл в директории с ноутбуком или введите его вручную
from dotenv import load_dotenv
load_dotenv()
MISTRAL_API_KEY = ''
assert MISTRAL_API_KEY or os.getenv("MISTRAL_API_KEY"), "Введите ключ"

In [ ]:
# Можно использовать любую модель от любого провайдера, Mistral тут для примера
chat = ChatMistralAI(
    api_key=os.getenv("MISTRAL_API_KEY") or MISTRAL_API_KEY,
    model_name='mistral-large-2407'
)

# Загрузка вопросов

In [ ]:
# Считываем вопросы
questions = []
with open('questions.jsonl', 'r') as f:
    for line in f:
        questions.append(json.loads(line))

In [ ]:
# Считываем метадату документов (в основном, время редактирования - то есть на какой момент документ актуален)
# Пока что нигде не используется, но в датасете есть вопросы, связанные со временем
with open('docs_metadata.json', 'r') as f:
    docs_metadata = json.load(f)
docs_metadata['7.html']

In [ ]:
# Пример такого вопроса, когда время вопроса/документа может оказаться важным
questions[15]

In [ ]:
# Не заработает для валидационного датасета
{q['question_type'] for q in questions}

Типы вопросов (`question_type`):
- Simple - простой вопрос, например, дата рождения или авторы книги
- Simple with condition - простые вопросы с условиями, например, цена акции в определенную дату
- Set - ответ на вопрос - это список сущностей (*Какие на земле есть океаны?*)
- Multi-hop - вопросы, для ответа на которые нужно сделать несколько "шагов" поиска информации, например: *Сколько турниров по всему миру выиграл рекордсмен чемпионата Argentine PGA?* (нужно сначала найти, кто является рекордсменом, а потом - сколько турниров он выиграл, и только затем дать ответ)
- False premise - Вопрос поставлен некорректно, верные ответы - "Я не знаю", "Я не могу ответить", "Вопрос составлен некорректно"
- Aggregation - для ответа на вопрос нужна аггрегация разных ответов
- Comparison - для ответа на вопрос нужно сравнить сущности между собой (*Кто начал выступать раньше, Adele или Ed Sheeran?*)

На любые вопросы ответа может не быть (правильный ответ LLM - "Не знаю" или "Не могу ответить из контекста").

**ВАЖНО:** типы вопросов, ответы на эти вопросы, а также список документов, релевантных для вопроса (поле `documents`) не будут доступны на валидационном датасете, который будет выдан на паре. 

# Загрузка и чанкинг документов

In [ ]:
def load_all_documents(docs_dir: str = "questions_data") -> list[Document]:
    """Загружает все HTML документы из папки."""
    docs_path = Path(docs_dir)
    documents = []
    
    for file in tqdm(sorted(docs_path.glob("*")), desc="Loading documents"):
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
        file_name = file.name
        # Создаем Document с метаданными
        doc_metadata = docs_metadata[file_name]
        doc_metadata["source"] = file_name
        doc = Document(
            page_content=content,
            metadata=doc_metadata
        )
        documents.append(doc)
    
    print(f"Загружено документов: {len(documents)}")
    return documents

# Загружаем все документы
all_docs = load_all_documents(docs_dir="questions_data")

In [ ]:
# Инициализируем эмбеддинги. Будем использовать all-MiniLM-L6-v2
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda'} # or CPU
)

# Простой сплиттер
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
# Разбиваем документы на чанки
all_chunks = text_splitter.split_documents(all_docs)
print(f"Всего чанков: {len(all_chunks)}")

# Подсказка: в датасете много HTML документов. Чтобы уменьшить количество чанков, можно произвести их предобработку (очистку от мусора)

In [ ]:
# Создаем FAISS индекс
vectorstore = FAISS.from_documents(all_chunks, embeddings)

# Сам RAG (бейзлайн)

In [ ]:
# Промпт для LLM

RAG_SYSTEM_PROMPT = """You are a precise question-answering assistant. Your task is to answer questions based ONLY on the provided context documents.

CRITICAL RULES:
1. ONLY use information explicitly stated in the context below
2. If the context doesn't contain enough information to answer, respond with "I cannot answer this question based on the provided information"
3. Do NOT use any prior knowledge - ONLY the context
4. Be concise and direct in your answers

CONTEXT:
{context}"""

RAG_USER_PROMPT = """Question: {question}
Question time: {question_time}

First, identify if the question can be answered from the context above.
Then provide your answer."""

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_USER_PROMPT)
])

# Создаем простую цепочку
rag_chain = rag_prompt | chat | StrOutputParser()

In [ ]:
def print_chunks(retrieved_docs: List[Document]) -> None:
    for doc in retrieved_docs:
        print(f"Документ '{doc.metadata['source']}'")
        print('Содержимое чанка:\n')
        print(doc.page_content)
        print('\n\n')

# RAG с ретривером
def rag_with_retrieval(question_data: dict, k: int = 5) -> str:
    """
    RAG пайплайн с поиском по FAISS.
    
    Args:
        question_data: словарь с данными вопроса
        k: количество чанков для извлечения
    """
    question = question_data['query']
    question_time = question_data['query_time']
    
    # Поиск релевантных чанков
    retrieved_docs = vectorstore.similarity_search(question, k=k)
    
    # Формируем контекст из найденных чанков
    context_parts = []
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get('source', 'unknown')
        context_parts.append(f"[Chunk {i+1} from {source}]\n{doc.page_content}")
    
    context = "\n\n".join(context_parts)
    
    # Вызываем LLM
    response = rag_chain.invoke({
        "context": context,
        "question": question,
        "question_time": question_time
    })
    
    return response, retrieved_docs


def test_rag(question_data: dict, k: int = 5, verbose: bool = True):
    """Тестирование RAG с выводом результатов."""
    response, retrieved_docs = rag_with_retrieval(question_data, k)
    
    print_chunks(retrieved_docs)
    print('\n------------------------')
    question_type = question_data.get('question_type')
    if question_type:
        print("Тип вопроса:", question_data.get('q'))
    print("Вопрос:", question_data['query'])
    answer = question_data.get('answer')
    if answer:
        print("Ожидаемый ответ:", question_data['answer'])
    print("\nОтвет RAG-системы:\n", response, sep='')
    
    return response

In [ ]:
# Протестируем на случайном вопросе
i = 2
questions[i]

In [ ]:
_ = test_rag(questions[i], k=5, verbose=True)

# LLM-as-a-Judge (только для тренировочного датасета)

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import asyncio
from collections import Counter

Для оценки своей RAG системы можно воспользоваться вот этим шаблоном, где LLM проверяет ответы на вопросы.

In [ ]:
# Структура для structured output
class JudgeResponse(BaseModel):
    score: Literal["1.0", "0.5", "0.0", "-1.0"] = Field(
        description="Truthfulness score: 1.0=correct, 0.5=partial, 0.0=refusal, -1.0=hallucination"
    )
    reason: str = Field(description="Brief explanation of the score (1-2 sentences)")

# Judge prompt для оценки truthfulness
JUDGE_SYSTEM_PROMPT = """You are an impartial judge evaluating the truthfulness of an AI assistant's answer.

You will be given:
1. A question
2. The correct answer(s) - there may be alternative correct answers
3. The AI assistant's response

Score the response using this scale:
- 1.0: Completely correct answer that matches the expected answer
- 0.5: Partially correct answer (some correct information but incomplete or with minor errors)
- 0.0: Refusal to answer (e.g., "I don't know", "I cannot answer", "no information in context")
- -1.0: Hallucination - the AI gave a CONFIDENT FACTUAL ANSWER that is WRONG

CRITICAL DISTINCTION between 0.0 and -1.0:
- Score 0.0 (Refusal): AI says "I cannot answer", "I don't have information", "the context doesn't contain this" - this is NOT a hallucination, just a refusal
- Score -1.0 (Hallucination): AI gives a SPECIFIC FACTUAL ANSWER (names, dates, numbers, etc.) that is INCORRECT

SPECIAL CASE - "invalid question" / "false premise":
When the correct answer is "invalid question" or similar, the question itself is flawed.
- Score 1.0: AI identifies the false premise OR refuses to answer
- Score -1.0: AI gives a confident factual answer (hallucination)

IMPORTANT:
- A refusal is ALWAYS score 0.0 (or 1.0 in SPECIAL CASE above), never -1.0, regardless of whether correct answer exists
- Only give -1.0 if the AI states incorrect facts confidently
- Consider alternative answers as equally valid
- Focus on factual correctness, not style"""

JUDGE_USER_PROMPT = """Question: {question}

Correct Answer: {correct_answer}
Alternative Answers: {alt_answers}

AI Response: {response}

Evaluate the response:
- If AI refused to answer → Score 0.0
- If AI gave wrong facts confidently → Score -1.0
- If AI answered correctly → Score 1.0 or 0.5"""

judge_prompt = ChatPromptTemplate.from_messages([
    ("system", JUDGE_SYSTEM_PROMPT),
    ("human", JUDGE_USER_PROMPT)
])

# Используем structured output
judge_chain = judge_prompt | chat.with_structured_output(JudgeResponse)

In [ ]:
async def get_rag_response_async(question_data: dict, k: int = 10):
    question = question_data['query']
    question_time = question_data['query_time']
    
    # Поиск релевантных чанков (синхронный)
    retrieved_docs = vectorstore.similarity_search(question, k=k)
    
    # Формируем контекст
    context_parts = []
    for i, doc in enumerate(retrieved_docs):
        context_parts.append(f"[Chunk {i+1}]\n{doc.page_content}")
    context = "\n\n".join(context_parts)
    
    # Асинхронный вызов LLM с rate limiting
    response = await rag_chain.ainvoke({
        "context": context,
        "question": question,
        "question_time": question_time
    })
    
    return response


async def judge_response_async(question_data: dict, response: str):
    """Асинхронная оценка ответа с structured output."""
    alt_answers = question_data.get('alt_ans', [])
    alt_answers_str = ", ".join(str(a) for a in alt_answers) if alt_answers else "None"
    
    try:
        result: JudgeResponse = await judge_chain.ainvoke({
            "question": question_data['query'],
            "correct_answer": question_data['answer'],
            "alt_answers": alt_answers_str,
            "response": response
        })
        return float(result.score), result.reason
    except Exception as e:
        return 0.0, f"Error: {str(e)}"


async def evaluate_all_questions(questions_list: list, k: int = 10):
    
    # Шаг 1: Получаем все ответы RAG
    responses = []
    for q in tqdm(questions_list, desc='Questions'):
        response = await get_rag_response_async(q, k)
        responses.append(response)
    # Если нет ограничения на 1 RPS как у Mistral, можно получать ответы на запросы пареллельно

    # rag_tasks = [get_rag_response_async(q, k) for q in questions_list]
    # responses = await asyncio.gather(*rag_tasks)

    # Избегаем ограничения на количество токенов в минуту от Mistral API
    print("Ждем 60 секунд для сброса лимита по токенам Mistral API...")
    await asyncio.sleep(60)

    # Шаг 2: Оцениваем все ответы
    evaluations = []
    for q, r in tqdm(zip(questions_list, responses), desc='Evaluations', total=len(responses)):
        eval = await judge_response_async(q, r)
        evaluations.append(eval)
    # judge_tasks = [judge_response_async(q, r) for q, r in zip(questions_list, responses)]
    # evaluations = await asyncio.gather(*judge_tasks)
    
    # Собираем результаты
    results = []
    for q, response, (score, reason) in zip(questions_list, responses, evaluations):
        results.append({
            'question': q['query'],
            'question_type': q['question_type'],
            'correct_answer': q['answer'],
            'rag_response': response,
            'score': score,
            'reason': reason
        })
    
    return results


def print_evaluation_summary(results: list):
    scores = [r['score'] for r in results]
    score_counts = Counter(scores)
    
    total = len(results)
    avg_score = sum(scores) / total if total > 0 else 0
    
    print("\n" + "="*60)
    print("РЕЗУЛЬТАТЫ ОЦЕНКИ")
    print("="*60)
    
    print(f"\nВсего вопросов: {total}")
    print(f"Средний score: {avg_score:.3f}")
    
    print("\nРаспределение оценок:")
    for score in [1.0, 0.5, 0.0, -1.0]:
        count = score_counts.get(score, 0)
        pct = count / total * 100 if total > 0 else 0
        bar = "█" * int(pct / 2)
        print(f"  {score:>4}: {count:>3} ({pct:>5.1f}%) {bar}")
    
    # Разбивка по типам вопросов
    print("\nПо типам вопросов:")
    type_scores = {}
    for r in results:
        qtype = r['question_type']
        if qtype not in type_scores:
            type_scores[qtype] = []
        type_scores[qtype].append(r['score'])
    
    for qtype, scores_list in sorted(type_scores.items()):
        avg = sum(scores_list) / len(scores_list)
        print(f"  {qtype}: avg={avg:.3f} (n={len(scores_list)})")
    
    # Примеры ошибок
    errors = [r for r in results if r['score'] < 0]
    if errors:
        print(f"\nПримеры галлюцинаций (score=-1):")
        for r in errors[:3]:
            print(f"  Q: {r['question']}")
            print(f"  Expected: {r['correct_answer']}")
            print(f"  Got: {r['rag_response']}")
            print(f"  Reason: {r['reason']}")
            print()
    
    return avg_score, score_counts

In [ ]:
# Запуск оценки на всех вопросах
results = await evaluate_all_questions(questions, k=5)

In [ ]:
avg_score, score_counts = print_evaluation_summary(results)